## Add coordinates to country file


In [ ]:
import pandas as pd
import os
from pathlib import Path

additional_coordinates = {
    "hk": [
        114.177216,
        22.302711,
    ],  # Hong kong
    "xk": [
        21.166191,
        42.667542,
    ],  # Kosovo
    "mo": [
        113.552971,
        22.210928,
    ],  # Macau
    "tw": [
        121.597366,
        25.105497,
    ],  # Taiwan
    "eh": [
        -13.162500,
        27.125286,
    ],  # Western Sahara
}
BASE_DIR = Path(os.getcwd()).resolve().parent.parent.parent

FRONTEND_DATA_DIR = BASE_DIR / "frontend/src/assets/data"
FRONTEND_STATIC_DIR = BASE_DIR / "frontend/public/static_data"

country_file = FRONTEND_DATA_DIR / "country.json"

centroids_file = FRONTEND_DATA_DIR / "country_centroids.geojson"

countries = pd.read_json(country_file)
centroids = pd.read_json(centroids_file)

centroids = pd.json_normalize(centroids["features"])
# Filter problematic entities - The ones with a toplevel ISO corresponding
# to a child entity.
mask = (
    centroids["properties.COUNTRY"] != centroids["properties.COUNTRYAFF"]
) & (centroids["properties.ISO"] == centroids["properties.AFF_ISO"])
centroids = centroids[~mask].copy()

cols = {"properties.ISO": "code", "geometry.coordinates": "coordinates"}
centroids = centroids[cols.keys()].rename(columns=cols)

centroids["code"] = centroids["code"].str.lower()

manual_centroids = pd.DataFrame(
    [{"code": k, "coordinates": v} for k, v in additional_coordinates.items()]
)
centroids = pd.concat(
    [
        centroids,
        manual_centroids[~manual_centroids["code"].isin(centroids["code"])],
    ],
    axis=0,
)
countries = countries.merge(centroids, on="code", how="left")

output_file = FRONTEND_STATIC_DIR / "country.json"
countries.to_json(output_file, orient="records", index=False, indent=2)